In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
import numpy as np
import pandas as pd
import re
import matplotlib.pyplot as plt
import matplotlib.style as style
import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelBinarizer
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.preprocessing.text import Tokenizer  
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.callbacks import ReduceLROnPlateau
style.use("seaborn")

In [ ]:
Hyper = {"num_words":1500 , "oov_token":'<OOV>' , "maxlen": 512 , "truncating":'post' , 'Embedding_dims' : 64 }

In [ ]:
DataSet = pd.read_csv("../input/imdb-dataset-of-50k-movie-reviews/IMDB Dataset.csv")

In [ ]:
DataSet.info()

In [ ]:
DataSet.describe()

In [ ]:
DataSet.head(3)

In [ ]:
sns.countplot(DataSet["sentiment"])

# PreProcessing Our Data

In [ ]:
def PreProcessing():
    Corpus = []
    for i in range( 0 , DataSet.shape[0]):
        review = re.sub('[^a-zA-Z]' , ' ' , DataSet["review"][i])
        review = review.lower()
        review = review.split()
        review = ' '.join(review)
        Corpus.append(review)
    return Corpus

In [ ]:
Corpus = PreProcessing()
#Make Catigoral Labels
Label =LabelBinarizer().fit_transform(DataSet['sentiment'])

In [ ]:
#Split Our Data
X_train ,X_test , y_train , y_test = train_test_split( Corpus,Label, test_size = 0.25 , random_state = 42)

In [ ]:
tokenizer = Tokenizer(num_words = Hyper["num_words"] ,oov_token = Hyper["oov_token"])
tokenizer.fit_on_texts(X_train)
word_index = tokenizer.word_index
sequance = tokenizer.texts_to_sequences(X_train)
pad_training_data = pad_sequences(sequance , maxlen=Hyper["maxlen"] , truncating = Hyper['truncating'])

In [ ]:
sequance = tokenizer.texts_to_sequences(X_train)
pad_training_data = pad_sequences(sequance , maxlen=Hyper["maxlen"], truncating =Hyper['truncating'])

In [ ]:
testing_sequences = tokenizer.texts_to_sequences(X_test)
testing_padded = pad_sequences(testing_sequences,maxlen=Hyper['maxlen']  , truncating = Hyper['truncating'])

In [ ]:
learning_rate_reduction = ReduceLROnPlateau(monitor='val_accuracy', patience = 2, verbose=1,factor=0.5, min_lr=0.00001)

In [ ]:
def My_Model():
    input_ = tf.keras.Input((Hyper['maxlen'],))
    
    x = tf.keras.layers.Embedding(len(tokenizer.word_index)+1,  Hyper['Embedding_dims'])(input_)
    
    x = tf.keras.layers.SpatialDropout1D(0.4)(x)
    
    x = tf.keras.layers.Conv1D(128 , 7 , activation = 'relu' , padding = 'same' , kernel_initializer='he_normal')(x)
    x = tf.keras.layers.Conv1D(65 , 7 , activation = 'relu' , padding = 'same' , kernel_initializer='he_normal')(x)
    
    x = tf.keras.layers.MaxPooling1D()(x)
    x = tf.keras.layers.BatchNormalization()(x)

    flatten = tf.keras.layers.GlobalMaxPooling1D()(x)
    x = tf.keras.layers.BatchNormalization()(flatten)
    
    x = tf.keras.layers.Dense(units = 32 , activation = 'relu' ,kernel_initializer='he_normal' )(x)
    
    x = tf.keras.layers.Dropout(0.3)(x)
    
    output_ = tf.keras.layers.Dense(units = 1 , activation = 'sigmoid')(x)
    
    return tf.keras.models.Model(inputs = [input_] , outputs = [output_])

In [ ]:
Model = My_Model()
Model.compile(optimizer = 'adam' , loss = 'binary_crossentropy' , metrics = ['accuracy'])
keras.utils.plot_model(Model)
print(Model.summary())

In [ ]:
History= Model.fit(pad_training_data , y_train , epochs = 10 , batch_size = 128 , validation_data = (testing_padded , y_test) , callbacks = [learning_rate_reduction])

In [ ]:
Model.evaluate(testing_padded , y_test)

In [ ]:
train_acc = History.history['accuracy']
train_loss = History.history['loss']
val_acc = History.history['val_accuracy']
val_loss = History.history['val_loss']
plt.plot(train_acc , 'go-',label = 'Training Accuracy')
plt.plot(val_acc , 'ro-', label = 'Testing Accuracy')
plt.plot(train_loss , 'mo-',label = 'Training Loss')
plt.plot(val_loss , 'co-',label = 'Testing Loss')
plt.legend()
plt.show()